#Enterprise Fleet Analytics Pipeline: Focuses on the business outcome (analytics) and the domain (fleet/logistics).

![logistics](logistics_project.png)

Download the data from the below gdrive and upload into the catalog
https://drive.google.com/drive/folders/1J3AVJIPLP7CzT15yJIpSiWXshu1iLXKn?usp=drive_link

##**1. Data Munging** -

####1. Visibily/Manually opening the file and capture couple of data patterns (Manual Exploratory Data Analysis)

Logistics source1 file:
- Format- CSV
- delimiter - comma
- line sep - new line
- header in file - yes
- missing columns
- extra columns
- data for format issues- id and age(expected int but received string)
- empty rows

Logistics source2 file:

- Format- CSV
- delimiter - comma
- line sep - new line
- header in file - yes
- missing columns
- data for format issues- id and age(expected int but received string)
- empty rows


####2. Programatically try to find couple of data patterns applying below EDA (File: logistics_source1)
1. Apply inferSchema and toDF to create a DF and analyse the actual data.
2. Analyse the schema, datatypes, columns etc.,
3. Analyse the duplicate records count and summary of the dataframe.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW shipment_temp_1
(
shipment_id INT,
first_name STRING,
last_name STRING,
age INT,
role STRING
)
USING CSV
OPTIONS
(
  path  "dbfs:/Volumes/usecase_data/logistics_proj_data/projdata/logistics_source1",
  header  "true",
  inferSchema  "false"

);
select * from shipment_temp_1;



In [0]:
##Use describe instead of schema or columns or dtypes or schema to understand the column names with datatypes
display(spark.sql("describe shipment_temp_1"))
shipment_df1=spark.sql("select * from shipment_temp_1")

##Understanding duplicates for individual columns
display(spark.sql("select count(*) from shipment_temp_1"))
display(spark.sql("select count(distinct shipment_id) from shipment_temp_1"))
display(shipment_df1.dropDuplicates(["shipment_id"]).count())
display(spark.sql("select count(distinct first_name) from shipment_temp_1"))
display(shipment_df1.dropDuplicates(["first_name"]).count())
display(spark.sql("select count(distinct last_name) from shipment_temp_1"))
display(shipment_df1.dropDuplicates(["last_name"]).count())

##Understanding duplicates for combination of columns is not possible becase only the first column with distinct will be considered
display(spark.sql("select count(*) from shipment_temp_1"))
##Below code doesnt work it applies distinct only on shipment_id
display(spark.sql("select count(distinct shipment_id, first_name, last_name) from shipment_temp_1"))
display(shipment_df1.dropDuplicates(["shipment_id","first_name","last_name"]).count())

##Understanding duplicate row is not possible directly
display(spark.sql("select count(*) from shipment_temp_1"))

##below code doesnt work as expected
display(spark.sql("select count(distinct *) from shipment_temp_1"))
display(shipment_df1.dropDuplicates().count())

#summary or describe helps us to understand statistical data understanding
#it shows there are nulls in few columns values, avg, stddev, min, max, percentile distribution
#to achive this we need to agg funtions But summary or describe can give this detail very easily

display(spark.sql("select count(shipment_id) as count,avg(shipment_id) avg, stddev(shipment_id), min(shipment_id), max(shipment_id) from shipment_temp_1"))

shipment_df1.summary().show()
        



###a. Passive Data Munging -  (File: logistics_source1  and logistics_source2)
Without modifying the data, identify:<br>
Shipment IDs that appear in both master_v1 and master_v2<br>
Records where:<br>
1. shipment_id is non-numeric
2. age is not an integer<br>

Count rows having:
3. fewer columns than expected
4. more columns than expected

In [0]:
#Create a Spark Session Object
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("MY Spark Session").getOrCreate()
print(spark)

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW shipment_temp_2
(
shipment_id INT,
first_name STRING,
last_name STRING,
age INT,
role STRING,
corrupt_record STRING
)
USING CSV
OPTIONS
(
  path  "dbfs:/Volumes/usecase_data/logistics_proj_data/projdata/logistics_source1",
  header  "true",
  inferSchema  "false",
  columnNameOfCorruptRecord "corrupt_record"

);
--Identifying the rows with less columns and additional columns
select * from shipment_temp_2 where corrupt_record is NOT NULL;

--identifying non int age from logistics_source1
--Here we dont have any string age boz.. while reading the file we read age as int

--INcase we read it as string and let all values.. direct casting is not possible .. we need to convert all strings manually to int using update

/*UPDATE person
SET age =
  CASE age
    WHEN 'one' THEN '1'
    WHEN 'two' THEN '2'
    WHEN 'three' THEN '3'
    WHEN 'ten' THEN '10'
    ELSE age
  END;*/



In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW shipment_temp1
(
shipment_id INT,
first_name STRING,
last_name STRING,
age INT,
role STRING
)
USING CSV
OPTIONS
(
  path  "dbfs:/Volumes/usecase_data/logistics_proj_data/projdata/logistics_source1",
  header  "true",
  inferSchema  "false"

);
CREATE OR REPLACE TEMPORARY VIEW shipment_temp2
(
shipment_id INT,
first_name STRING,
last_name STRING,
age INT,
role STRING,
hub_location STRING,
vehicle_type STRING
)
USING CSV
OPTIONS
(
  path  "dbfs:/Volumes/usecase_data/logistics_proj_data/projdata/logistics_source2",
  header  "true",
  inferSchema  "false"
);

In [0]:
%sql
SELECT t1.shipment_id FROM  shipment_temp1 t1 join shipment_temp2 t2 on t1.shipment_id = t2.shipment_id 

###**b. Active Data Munging** File: logistics_source1 and logistics_source2

#####1.Combining Data + Schema Merging (Structuring)
1. Read both files without enforcing schema
2. Align them into a single canonical schema: shipment_id,
first_name,
last_name,
age,
role,
hub_location,
vehicle_type,
data_source
3. Add data_source column with values as: system1, system2 in the respective dataframes

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW shipment_temp_1
(
shipment_id STRING,
first_name STRING,
last_name STRING,
age STRING,

role STRING,
hub_location STRING,
vehicle_type STRING
)
USING CSV
OPTIONS
(
  path  "dbfs:/Volumes/usecase_data/logistics_proj_data/projdata/logistics_source1",
  header  "true",
  inferSchema  "false"
);
CREATE  OR REPLACE TEMPORARY VIEW shipment_temp_2
(
shipment_id STRING,
first_name STRING,
last_name STRING,
age STRING,
role STRING,
hub_location STRING,
vehicle_type STRING
)
USING CSV
OPTIONS
(
  path  "dbfs:/Volumes/usecase_data/logistics_proj_data/projdata/logistics_source2",
  header  "true",
  inferSchema  "false"

);

Create or replace temp view shipment_temp_raw as select *,'source1' as data_source from shipment_temp_1  union all select  *,'source2' as data_source from shipment_temp_2 ;
drop table if exists usecase_data.logistics_sql_data.shipment_raw_table;
create table if not exists usecase_data.logistics_sql_data.shipment_raw_table as select * from shipment_temp_raw;
select * from shipment_temp_raw;

In [0]:

%sql
CREATE OR REPLACE TEMPORARY VIEW master_city_list( 
city_name STRING,
country STRING,
latitude DECIMAL(10,6),
longitude DECIMAL(10,6))
USING CSV
OPTIONS(
header "True",
path "dbfs:/Volumes/usecase_data/logistics_proj_data/projdata/Master_City_List.csv",
inferSchema "False"
);
DROP table IF EXISTS usecase_data.logistics_sql_data.master_city_table;
CREATE TABLE IF NOT EXISTS usecase_data.logistics_sql_data.master_city_table AS SELECT * FROM master_city_list;

#####2. Cleansing, Scrubbing: 
Cleansing (removal of unwanted datasets)<br>
1. Mandatory Column Check - Drop any record where any of the following columns is NULL:shipment_id, role<br>
2. Name Completeness Rule - Drop records where both of the following columns are NULL: first_name, last_name<br>
3. Join Readiness Rule - Drop records where the join key is null: shipment_id<br>

Scrubbing (convert raw to tidy)<br>
4. Age Defaulting Rule - Fill NULL values in the age column with: -1<br>
5. Vehicle Type Default Rule - Fill NULL values in the vehicle_type column with: UNKNOWN<br>
6. Invalid Age Replacement - Replace the following values in age:
"ten" to -1
"" to -1<br>
7. Vehicle Type Normalization - Replace inconsistent vehicle types: 
truck to LMV
bike to TwoWheeler

In [0]:
%sql
create or replace temp view shipment_temp_raw1
as 
select 
shipment_id,
first_name,
last_name,
Case when coalesce(age,'-1') = 'ten' or coalesce(age,'-1') = '' then '-1'  else age end as age,
role,
case when hub_location = 'Additionalcolumn' then NULL else hub_location end as hub_location,
coalesce(vehicle_type,'UNKNOWN') AS vehicle_type,
data_source
from shipment_temp_raw 
where shipment_id is not null and 
role is not null 
and ( first_name is not null or last_name is not null)

####3. Standardization, De-Duplication and Replacement / Deletion of Data to make it in a usable format

Creating shipments Details data Dataframe creation <br>
1. Create a DF by Reading Data from logistics_shipment_detail.json
2. As this data is a clean json data, it doesn't require any cleansing or scrubbing.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW  logistics_shipment_temp 
(shipment_id INT,
order_id STRING,
source_city STRING,
destination_city STRING,
shipment_status STRING,
cargo_type STRING,
vehicle_type STRING,
payment_mode STRING,
shipment_weight_kg FLOAT,
shipment_cost FLOAT,
shipment_date STRING)
USING JSON
OPTIONS
(
  path  "dbfs:/Volumes/usecase_data/logistics_proj_data/projdata/logistics_shipment_detail_3000.json",
  multiLine "True");
  DROP TABLE IF EXISTS  usecase_data.logistics_sql_data.logistics_shipment_raw_table;
CREATE TABLE IF NOT EXISTS usecase_data.logistics_sql_data.logistics_shipment_raw_table AS SELECT * FROM logistics_shipment_temp;


Standardizations:<br>

1. Add a column<br> 
Source File: DF of logistics_shipment_detail_3000.json<br>: domain as 'Logistics',  current timestamp 'ingestion_timestamp' and 'False' as 'is_expedited'
2. Column Uniformity: 
role - Convert to lowercase<br>
Source File: DF of merged(logistics_source1 & logistics_source2)<br>
vehicle_type - Convert values to UPPERCASE<br>
Source Files: DF of logistics_shipment_detail_3000.json
hub_location - Convert values to initcap case<br>
Source Files: DF of merged(logistics_source1 & logistics_source2)<br>
3. Format Standardization:<br>
Source Files: DF of logistics_shipment_detail_3000.json<br>
Convert shipment_date to yyyy-MM-dd<br>
Ensure shipment_cost has 2 decimal precision<br>
4. Data Type Standardization<br>
Standardizing column data types to fix schema drift and enable mathematical operations.<br>
Source File: DF of merged(logistics_source1 & logistics_source2) <br>
age: Cast String to Integer<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
shipment_weight_kg: Cast to Double<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
is_expedited: Cast to Boolean<br>
5. Naming Standardization <br>
Source File: DF of merged(logistics_source1 & logistics_source2)<br>
Rename: first_name to staff_first_name<br>
Rename: last_name to staff_last_name<br>
Rename: hub_location to origin_hub_city<br>
6. Reordering columns logically in a better standard format:<br>
Source File: DF of Data from all 3 files<br>
shipment_id (Identifier), staff_first_name (Dimension)staff_last_name (Dimension), role (Dimension), origin_hub_city (Location), shipment_cost (Metric), ingestion_timestamp (Audit)

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW  logistics_shipment_temp_1
AS SELECT
shipment_id ,
order_id ,
source_city ,
destination_city ,
shipment_status ,
cargo_type ,
upper(vehicle_type) AS vehicle_type ,
payment_mode ,
round(cast(shipment_weight_kg as double),2) as shipment_weight_kg,
round(cast(shipment_cost as double),2) as shipment_cost ,
to_date(shipment_date,'yy-MM-dd') as shipment_date,
'Logistics' AS domain, 
current_timestamp() as  ingestion_timestamp,
False as is_expedited
FROM logistics_shipment_temp;
--select * from  logistics_shipment_temp_1

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW shipment_temp_raw2
AS SELECT 
cast(shipment_id as int)as shipment_id,
first_name as staff_first_name,
last_name as staff_last_name,
cast(age as int) as age,
LOWER(role) AS role,
initcap(hub_location) as origin_hub_city,
vehicle_type,
data_source
 FROM shipment_temp_raw1 where shipment_id != 'ten';
select * from shipment_temp_raw2

Deduplication:
1. Apply Record Level De-Duplication
2. Apply Column Level De-Duplication (Primary Key Enforcement)

In [0]:
%sql
--row level distinct
create or replace temporary view shipment_temp_raw3 as select distinct * from shipment_temp_raw2;
--column level distinct
create or replace temporary view shipment_temp_raw4 as  select * from shipment_temp_raw3 
qualify row_number() over(partition by shipment_id order by data_source asc)=1




##2. Data Enrichment - Detailing of data
Makes your data rich and detailed <br>

###### Adding of Columns (Data Enrichment)
*Creating new derived attributes to enhance traceability and analytical capability.*

**1. Add Audit Timestamp (`load_dt`)**
Source File: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** We need to track exactly when this record was ingested into our Data Lakehouse for auditing purposes.
* **Action:** Add a column `load_dt` using the function `current_timestamp()`.

**2. Create Full Name (`full_name`)**
Source File: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** The reporting dashboard requires a single field for the driver's name instead of separate columns.
* **Action:** Create `full_name` by concatenating `first_name` and `last_name` with a space separator.
* **Result:** "Rajesh" + " " + "Kumar" -> **"Rajesh Kumar"**

**3. Define Route Segment (`route_segment`)**
Source File: DF of logistics_shipment_detail_3000.json<br>
* **Scenario:** The logistics team wants to analyze performance based on specific transport lanes (Source to Destination).
* **Action:** Combine `source_city` and `destination_city` with a hyphen.
* **Result:** "Chennai" + "-" + "Pune" -> **"Chennai-Pune"**

**4. Generate Vehicle Identifier (`vehicle_identifier`)**
Source File: DF of logistics_shipment_detail_3000.json<br>
* **Scenario:** We need a unique tracking code that immediately tells us the vehicle type and the shipment ID.
* **Action:** Combine `vehicle_type` and `shipment_id` to create a composite key.
* **Result:** "Truck" + "_" + "500001" -> **"Truck_500001"**

###### Remove/Eliminate (drop, select, selectExpr)
*Excluding unnecessary or redundant columns to optimize storage and privacy.*<br>
Source File: DF of logistics_source1 and logistics_source2<br>

**1. Remove Redundant Name Columns**
* **Scenario:** Since we have already created the `full_name` column in the Enrichment step, the individual name columns are now redundant and clutter the dataset.
* **Action:** Drop the `first_name` and `last_name` columns.
* **Logic:** `df.drop("first_name", "last_name")`

In [0]:
%sql
create or replace temporary view shipment_temp_raw5
as select 
shipment_id,
trim(concat_ws('',staff_first_name,' ',staff_last_name)) as full_name,
age,
role,
origin_hub_city,
vehicle_type,
data_source,
current_timestamp() as load_dt
from shipment_temp_raw4;

###### Deriving of Columns (Time Intelligence)
*Extracting temporal features from dates to enable period-based analysis and reporting.*<br>
Source File: logistics_shipment_detail_3000.json<br>
**1. Derive Shipment Year (`shipment_year`)**
* **Scenario:** Management needs an annual performance report to compare growth year-over-year.
* **Action:** Extract the year component from `shipment_date`.
* **Result:** "2024-04-23" -> **2024**

**2. Derive Shipment Month (`shipment_month`)**
* **Scenario:** Analysts want to identify seasonal peaks (e.g., increased volume in December).
* **Action:** Extract the month component from `shipment_date`.
* **Result:** "2024-04-23" -> **4** (April)

**3. Flag Weekend Operations (`is_weekend`)**
* **Scenario:** The Operations team needs to track shipments handled during weekends to calculate overtime pay or analyze non-business day capacity.
* **Action:** Flag as **'True'** if the `shipment_date` falls on a Saturday or Sunday.

**4. Flag shipment status (`is_expedited`)**
* **Scenario:** The Operations team needs to track shipments is IN_TRANSIT or DELIVERED.
* **Action:** Flag as **'True'** if the `shipment_status` IN_TRANSIT or DELIVERED.

###### Enrichment/Business Logics (Calculated Fields)
*Deriving new metrics and financial indicators using mathematical and date-based operations.*<br>
Source File: logistics_shipment_detail_3000.json<br>

**1. Calculate Unit Cost (`cost_per_kg`)**
* **Scenario:** The Finance team wants to analyze the efficiency of shipments by determining the cost incurred per unit of weight.
* **Action:** Divide `shipment_cost` by `shipment_weight_kg`.
* **Logic:** `shipment_cost / shipment_weight_kg`

**2. Track Shipment Age (`days_since_shipment`)**
* **Scenario:** The Operations team needs to monitor how long it has been since a shipment was dispatched to identify potential delays.
* **Action:** Calculate the difference in days between the `current_date` and the `shipment_date`.
* **Logic:** `datediff(current_date(), shipment_date)`

**3. Compute Tax Liability (`tax_amount`)**
* **Scenario:** For invoicing and compliance, we must calculate the Goods and Services Tax (GST) applicable to each shipment.
* **Action:** Calculate 18% GST on the total `shipment_cost`.
* **Logic:** `shipment_cost * 0.18`

##### Splitting & Merging/Melting of Columns
*Reshaping columns to extract hidden values or combine fields for better analysis.*<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
**1. Splitting (Extraction)**
*Breaking one column into multiple to isolate key information.*
* **Split Order Code:**
  * **Action:** Split `order_id` ("ORD100000") into two new columns:
    * `order_prefix` ("ORD")
    * `order_sequence` ("100000")
* **Split Date:**
  * **Action:** Split `shipment_date` into three separate columns for partitioning:
    * `ship_year` (2024)
    * `ship_month` (4)
    * `ship_day` (23)

**2. Merging (Concatenation)**
*Combining multiple columns into a single unique identifier or description.*
* **Create Route ID:**
  * **Action:** Merge `source_city` ("Chennai") and `destination_city` ("Pune") to create a descriptive route key:
    * `route_lane` ("Chennai->Pune")

In [0]:
%sql
create or replace temporary view logistics_shipment_temp_2 as
select 
shipment_id,
order_id ,
source_city ,
destination_city ,
shipment_status ,
cargo_type ,
vehicle_type ,
payment_mode ,
shipment_weight_kg,
shipment_cost ,
shipment_date,
domain, 
ingestion_timestamp,
case when shipment_status in ('IN_TRANSIT','DELIVERED') then True else False end as is_expedited,
concat_ws('',source_city,'-',destination_city) as route_segment,
concat_ws('',vehicle_type,'_',shipment_id) as vehicle_identifier,
year(shipment_date) as shipment_year,
month(shipment_date) as shipment_month,
case when 
dayofweek(shipment_date) in(1,7) then True else False end as is_weekend,
round(try_divide(shipment_cost,shipment_weight_kg),2) as cost_per_kg,
datediff(current_date(), shipment_date) as days_since_shipment,
round(shipment_cost * 0.18,2) as tax_amount,
substring(order_id,1,3) as order_prefix,
substring(order_id,4,len(order_id)) as order_sequence,
year(shipment_date) as ship_year,
month(shipment_date) as ship_month,
day(shipment_date) as ship_day,
concat_ws('',source_city,'->',destination_city) as route_lane,
case when shipment_cost > 50000 then True else False end as is_high_value
from logistics_shipment_temp_1;



## 3. Data Customization & Processing - Application of Tailored Business Specific Rules

### **UDF1: Complex Incentive Calculation**
**Scenario:** The Logistics Head wants to calculate a "Performance Bonus" for drivers based on tenure and role complexity.

**Action:** Create a Python function `calculate_bonus(role, age)` and register it as a Spark UDF.

**Logic:**
* **IF** `Role` == 'Driver' **AND** `Age` > 50:
  * `Bonus` = 15% of Salary (Reward for Seniority)
* **IF** `Role` == 'Driver' **AND** `Age` < 30:
  * `Bonus` = 5% of Salary (Encouragement for Juniors)
* **ELSE**:
  * `Bonus` = 0

**Result:** A new derived column `projected_bonus` is generated for every row in the dataset.

---

### **UDF2: PII Masking (Privacy Compliance)**
**Scenario:** For the analytics dashboard, we must hide the full identity of the staff to comply with privacy laws (GDPR/DPDP), while keeping names recognizable for internal managers.

**Business Rule:** Show the first 2 letters, mask the middle characters with `****`, and show the last letter.

**Action:** Create a UDF `mask_identity(name)`.

**Example:**
* **Input:** `"Rajesh"`
* **Output:** `"Ra****h"`
<br>
**Note: Convert the above udf logic to inbult function based transformation to ensure the performance is improved.**

In [0]:
%sql
create or replace temporary view shipment_temp_raw6
as 
select
shipment_id,
concat(substring(full_name,1,2),repeat('*',len(full_name)-3),substring(full_name,-1)) as staff_full_name,
age,
role,
origin_hub_city,
vehicle_type,
data_source,
current_timestamp() as load_dt,
case 
when role='driver' and age > 50 then 0.15
when role='driver' and age < 30 then 0.05
else 0
end as projected_bonus
from shipment_temp_raw5;



## 4. Data Core Curation & Processing (Pre-Wrangling)
*Applying business logic to focus, filter, and summarize data before final analysis.*

**1. Select (Projection)**<br>
Source Files: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** The Driver App team only needs location data, not sensitive HR info.
* **Action:** Select only `first_name`, `role`, and `hub_location`.

**2. Filter (Selection)**<br>
Source File: DF of json<br>
* **Scenario:** We need a report on active operational problems.
* **Action:** Filter rows where `shipment_status` is **'DELAYED'** or **'RETURNED'**.
* **Scenario:** Insurance audit for senior staff.
* **Action:** Filter rows where `age > 50`.

**3. Derive Flags & Columns (Business Logic)**<br>
Source File: DF of json<br>
* **Scenario:** Identify high-value shipments for security tracking.
* **Action:** Create flag `is_high_value` = **True** if `shipment_cost > 50,000`.
* **Scenario:** Flag weekend operations for overtime calculation.
* **Action:** Create flag `is_weekend` = **True** if day is Saturday or Sunday.

**4. Format (Standardization)**<br>
Source File: DF of json<br>
* **Scenario:** Finance requires readable currency formats.
* **Action:** Format `shipment_cost` to string like **"₹30,695.80"**.
* **Scenario:** Standardize city names for reporting.
* **Action:** Format `source_city` to Uppercase (e.g., "chennai" → **"CHENNAI"**).

**5. Group & Aggregate (Summarization)**<br>
Source Files: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** Regional staffing analysis.
* **Action:** Group by `hub_location` and **Count** the number of staff.
* **Scenario:** Fleet capacity analysis.
* **Action:** Group by `vehicle_type` and **Sum** the `shipment_weight_kg`.

**6. Sorting (Ordering)**<br>
Source File: DF of json<br>
* **Scenario:** Prioritize the most expensive shipments.
* **Action:** Sort by `shipment_cost` in **Descending** order.
* **Scenario:** Organize daily dispatch schedule.
* **Action:** Sort by `shipment_date` (Ascending) then `priority_flag` (Descending).

**7. Limit (Top-N Analysis)**<br>
Source File: DF of json<br>
* **Scenario:** Dashboard snapshot of critical delays.
* **Action:** Filter for 'DELAYED', Sort by Cost, and **Limit to top 10** rows.

In [0]:
%sql
drop table if exists usecase_data.logistics_sql_data.driver_app_data;

CREATE Table if not EXISTS 
usecase_data.logistics_sql_data.driver_app_data
as 
select staff_full_name, role, origin_hub_city from shipment_temp_raw6;

DROP TABLE IF EXISTS usecase_data.logistics_sql_data.active_operational_problem_data;

CREATE TABLE IF NOT EXISTS usecase_data.logistics_sql_data.active_operational_problem_data
as select * from logistics_shipment_temp_2 where shipment_status IN ('DELAYED','RETURNED');

drop table if exists usecase_data.logistics_sql_data.senior_insurance_audit;

CREATE Table if not EXISTS 
usecase_data.logistics_sql_data.senior_insurance_audit
as 
select * from shipment_temp_raw6 where age >50;



In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW logistics_shipment_temp_3 AS
select 
shipment_id as log_shipment_id,
order_id ,
upper(source_city) as source_city ,
destination_city ,
shipment_status ,
cargo_type ,
vehicle_type as shipment_vehicle_type,
payment_mode ,
shipment_weight_kg,
shipment_cost,
concat('₹',cast(shipment_cost as string)) as shipment_cost_inr ,
shipment_date,
domain, 
ingestion_timestamp,
is_expedited,
route_segment,
vehicle_identifier,
shipment_year,
shipment_month,
is_weekend,
cost_per_kg,
days_since_shipment,
tax_amount,
order_prefix,
order_sequence,
ship_year,
ship_month,
ship_day,
route_lane,
is_high_value
from logistics_shipment_temp_2;

In [0]:
%sql
drop table if exists usecase_data.logistics_sql_data.regional_staffing_analysis;
CREATE TABLE IF NOT EXISTS usecase_data.logistics_sql_data.regional_staffing_analysis as 
select origin_hub_city, count(*) as staff_count from shipment_temp_raw6 group by origin_hub_city;

drop table if exists usecase_data.logistics_sql_data.fleet_capacity_analysis;
    
CREATE TABLE IF NOT EXISTS usecase_data.logistics_sql_data.fleet_capacity_analysis as 
select shipment_vehicle_type, sum(shipment_weight_kg) as total_weight from logistics_shipment_temp_3 group by shipment_vehicle_type;


    


In [0]:
%sql
--daily dispatch schedule based on the cost priority
drop table if exists usecase_data.logistics_sql_data.daily_dispatch_schedule;
CREATE TABLE IF NOT EXISTS usecase_data.logistics_sql_data.daily_dispatch_schedule AS 
SELECT
* FROM logistics_shipment_temp_3 order by shipment_date asc, shipment_cost desc;


In [0]:
%sql
drop table if exists usecase_data.logistics_sql_data.critical_delays;
CREATE TABLE IF NOT EXISTS usecase_data.logistics_sql_data.critical_delays as
select * from  logistics_shipment_temp_3 where shipment_status='DELAYED' order by shipment_date asc,shipment_cost desc limit 10

## 5. Data Wrangling - Transformation & Analytics
*Combining, modeling, and analyzing data to answer complex business questions.*

### **1. Joins**
Source Files:<br>
Left Side (staff_df):<br> DF of logistics_source1 & logistics_source2<br>
Right Side (shipments_df):<br> DF of logistics_shipment_detail_3000.json<br>
#### **1.1 Frequently Used Simple Joins (Inner, Left)**
* **Inner Join (Performance Analysis):**
  * **Scenario:** We only want to analyze *completed work*. Connect Staff to the Shipments they handled.
  * **Action:** Join `staff_df` and `shipments_df` on `shipment_id`.
  * **Result:** Returns only rows where a staff member is assigned to a valid shipment.
* **Left Join (Idle Resource check):**
  * **Scenario:** Find out which staff members are currently *idle* (not assigned to any shipment).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right) on `shipment_id`. Filter where `shipments_df.shipment_id` is NULL.

#### **1.2 Infrequent Simple Joins (Self, Right, Full, Cartesian)**
* **Self Join (Peer Finding):**
  * **Scenario:** Find all pairs of employees working in the same `hub_location`.
  * **Action:** Join `staff_df` to itself on `hub_location`, filtering where `staff_id_A != staff_id_B`.
* **Right Join (Orphan Data Check):**
  * **Scenario:** Identify shipments in the system that have *no valid driver* assigned (Data Integrity Issue).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right). Focus on NULLs on the left side.
* **Full Outer Join (Reconciliation):**
  * **Scenario:** A complete audit to find *both* idle drivers AND unassigned shipments in one view.
  * **Action:** Perform a Full Outer Join on `shipment_id`.
* **Cartesian/Cross Join (Capacity Planning):**
  * **Scenario:** Generate a schedule of *every possible* driver assignment to *every* pending shipment to run an optimization algorithm.
  * **Action:** Cross Join `drivers_df` and `pending_shipments_df`.

#### **1.3 Advanced Joins (Semi and Anti)**
* **Left Semi Join (Existence Check):**
  * **Scenario:** "Show me the details of Drivers who have *at least one* shipment." (Standard filtering).
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_semi")`.
  * **Benefit:** Performance optimization; it stops scanning the right table once a match is found.
* **Left Anti Join (Negation Check):**
  * **Scenario:** "Show me the details of Drivers who have *never* touched a shipment."
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_anti")`.

### **2. Lookup**<br>
Source File: DF of logistics_source1 and logistics_source2 (merged into Staff DF)<br>
* **Scenario:** Validation. Check if the `hub_location` in the staff file exists in the corporate `Master_City_List`.
* **Action:** Compare values against a reference list.

### **3. Lookup & Enrichment**<br>
Source File: DF of logistics_source1 and logistics_source2 (merged into Staff DF)<br>
* **Scenario:** Geo-Tagging.
* **Action:** Lookup `hub_location` ("Pune") in a Master Latitude/Longitude table and enrich the dataset by adding `lat` and `long` columns for map plotting.

### **4. Schema Modeling (Denormalization)**<br>
Source Files: DF of All 3 Files (logistics_source1, logistics_source2, logistics_shipment_detail_3000.json)<br>
* **Scenario:** Creating a "Gold Layer" Table for PowerBI/Tableau.
* **Action:** Flatten the Star Schema. Join `Staff`, `Shipments`, and `Vehicle_Master` into one wide table (`wide_shipment_history`) so analysts don't have to perform joins during reporting.

### **5. Windowing (Ranking & Trends)**<br>
Source Files:<br>
DF of logistics_source2: Provides hub_location (Partition Key).<br>
logistics_shipment_detail_3000.json: Provides shipment_cost (Ordering Key)<br>
* **Scenario:** "Who are the Top 3 Drivers by Cost in *each* Hub?"
* **Action:**
  1. Partition by `hub_location`.
  2. Order by `total_shipment_cost` Descending.
  3. Apply `dense_rank()` and `row_number()
  4. Filter where `rank or row_number <= 3`.

### **6. Analytical Functions (Lead/Lag)**<br>
Source File: <br>
DF of logistics_shipment_detail_3000.json<br>
* **Scenario:** Idle Time Analysis.
* **Action:** For each driver, calculate the days elapsed since their *previous* shipment.

### **7. Set Operations**<br>
Source Files: DF of logistics_source1 and logistics_source2<br>
* **Union:** Combining `Source1` (Legacy) and `Source2` (Modern) into one dataset (Already done in Active Munging).
* **Intersect:** Identifying Staff IDs that appear in *both* Source 1 and Source 2 (Duplicate/Migration Check).
* **Except (Difference):** Identifying Staff IDs present in Source 2 but *missing* from Source 1 (New Hires).

### **8. Grouping & Aggregations (Advanced)**<br>
Source Files:<br>
DF of logistics_source2: Provides hub_location and vehicle_type (Grouping Dimensions).<br>
DF of logistics_shipment_detail_3000.json: Provides shipment_cost (Aggregation Metric).<br>
* **Scenario:** The CFO wants a subtotal report at multiple levels:
  1. Total Cost by Hub.
  2. Total Cost by Hub AND Vehicle Type.
  3. Grand Total.
* **Action:** Use `cube("hub_location", "vehicle_type")` or `rollup()` to generate all these subtotals in a single query.

In [0]:
%sql
drop table if exists usecase_data.logistics_sql_data.staff_with_shipments;
CREATE TABLE if not exists usecase_data.logistics_sql_data.staff_with_shipments AS
SELECT
*
from shipment_temp_raw6 t1 
inner join logistics_shipment_temp_3 t2 
on t1.shipment_id=t2.log_shipment_id;

drop table if exists usecase_data.logistics_sql_data.idle_staff;
CREATE TABLE if not exists usecase_data.logistics_sql_data.idle_staff
as 
select t1.* 
from shipment_temp_raw6 t1 
left join logistics_shipment_temp_3 t2 
on t1.shipment_id=t2.log_shipment_id 
where t2.log_shipment_id is null;



In [0]:
%sql
select t1.shipment_id,t1.origin_hub_city,t2.shipment_id,t2.origin_hub_city from shipment_temp_raw6 t1 join shipment_temp_raw6 t2 on t1.origin_hub_city=t2.origin_hub_city where t1.shipment_id!=t2.shipment_id;

In [0]:
%sql
drop table if exists usecase_data.logistics_sql_data.invalid_driver_to_shipments;
CREATE TABLE if not exists usecase_data.logistics_sql_data.invalid_driver_to_shipments
as 
select t2.* from shipment_temp_raw6 t1 right join logistics_shipment_temp_3 t2 on t1.shipment_id=t2.log_shipment_id where t1.shipment_id is null


In [0]:
%sql
select * from shipment_temp_raw6 t1 full outer join logistics_shipment_temp_3 t2 on t1.shipment_id=t2.log_shipment_id where t1.shipment_id is null or t2.log_shipment_id is null


In [0]:
%sql
WITH driver_tl1 AS (
    SELECT *
    FROM shipment_temp_raw6
    WHERE role = 'driver'
),
pending_shipment AS (
    SELECT *
    FROM logistics_shipment_temp_3
    WHERE shipment_status = 'DELAYED'
)
SELECT *
FROM driver_tl1 t1
JOIN pending_shipment t2
ON 1 = 1;



In [0]:
%sql
--left semi join
select * from shipment_temp_raw6 t1 left semi join logistics_shipment_temp_3 t2 on t1.shipment_id=t2.log_shipment_id;
--left anti join
select * from shipment_temp_raw6 t1 left anti join logistics_shipment_temp_3 t2 on t1.shipment_id=t2.log_shipment_id;


In [0]:
%sql
select t1.origin_hub_city from shipment_temp_raw6 t1 left semi join master_city_list t2 on t1.origin_hub_city=t2.city_name;


In [0]:
%sql
DROP TABLE IF EXISTS usecase_data.logistics_sql_data.geo_tagged_staff_data;
CREATE TABLE IF NOT EXISTS usecase_data.logistics_sql_data.geo_tagged_staff_data AS
select t1.*,t2.latitude,t2.longitude from shipment_temp_raw6 t1 left join master_city_list t2 on t1.origin_hub_city=t2.city_name;

In [0]:
%sql
drop table if exists usecase_data.logistics_sql_data.wide_shipment_history;
create table if not exists usecase_data.logistics_sql_data.wide_shipment_history as
select * from logistics_shipment_temp_3 t1 left join shipment_temp_raw6 t2 on t1.log_shipment_id=t2.shipment_id left join master_city_list t3 on t2.origin_hub_city=t3.city_name

In [0]:
%sql
drop table if exists usecase_data.logistics_sql_data.top_driver_list;
create table if not exists usecase_data.logistics_sql_data.top_driver_list as
with driver_list as (
select * from usecase_data.logistics_sql_data.wide_shipment_history where role='driver')
select * from driver_list qualify row_number() over(partition by origin_hub_city order by shipment_cost desc) <=3


In [0]:
%sql
select *,date_diff(shipment_date,(lag(shipment_date) over(partition by log_shipment_id order by shipment_date asc))) as elapsed_days from logistics_shipment_temp_3 

In [0]:
%sql
drop table if exists usecase_data.logistics_sql_data.migrant_staff;
create table if not exists usecase_data.logistics_sql_data.migrant_staff as
select shipment_id from shipment_temp_1 intersect select shipment_id from shipment_temp_2;
drop table if exists usecase_data.logistics_sql_data.new_hires;
create table if not exists usecase_data.logistics_sql_data.new_hires as
select shipment_id from shipment_temp_2 except select shipment_id from shipment_temp_1;

In [0]:
%sql
drop table if exists usecase_data.logistics_sql_data.multi_level_report;
create table if not exists usecase_data.logistics_sql_data.multi_level_report as
select sum(shipment_cost) as total_cost, origin_hub_city, vehicle_type from usecase_data.logistics_sql_data.staff_with_shipments group by cube(origin_hub_city,vehicle_type)

In [0]:
multilevel_subtotal_report=inner_joined_df.cube("origin_hub_city","vehicle_type").agg(sum("shipment_cost").alias("total_cost"))
display(multilevel_subtotal_report)

##6. Data Persistance (LOAD)-> Data Publishing & Consumption<br>

Store the inner joined, lookup and enrichment, Schema Modeling, windowing, analytical functions, set operations, grouping and aggregation data into the delta tables.

##7.Take the copy of the above notebook and try to write the equivalent SQL for which ever applicable.